In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import seaborn as sns
import matplotlib.pyplot as plt
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
#import tokenization

In [ ]:
test=pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='ISO-8859-1')
train=pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='ISO-8859-1')

In [ ]:
train.head(5)

In [ ]:
train.isnull().sum()

In [ ]:
train.head(2)

In [ ]:
x=train.Sentiment
x

In [ ]:
plt.figure(figsize=(5,10))
sns.countplot(train.Sentiment)

In [ ]:
X=train['OriginalTweet']

In [ ]:
y=train['Sentiment']

In [ ]:
import nltk
lst_stopwords = nltk.corpus.stopwords.words("english")
lst_stopwords

In [ ]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [ ]:
def bert_encode(texts,tokenizer,max_len=512):
    all_tokens=[]
    all_masks=[]
    all_segments=[]
    for text in texts:
        text=tokenizer.tokenize(text)
        text=text[:max_len-2]
        input_sequence=['[CLS]']+text+['[SEP]']
        pad_len=max_len-len(input_sequence)
        tokens=tokenizer.convert_tokens_to_ids(input_sequence)
        tokens+=[0]*pad_len
        pad_masks[1]*len(input_sequence)+[0]*pad_len
        segment_ids=[0]*max_len
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segment.append(segment_ids)
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [ ]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model